# Aplicação: Reconhecimento de Entidades Nomeadas

https://www.tensorflow.org/datasets/catalog/conll2003


CoNLL 2003
This dataset includes 1,393 English and 909 German news articles. The English-language corpus is free, but the German corpus comes at $75, unfortunately. This is the only corpus that costs something in this post. To build the English-language corpus you need the RCV1 Reuters corpus. You will obtain access a couple days after submitting the organisational and individual agreement at no charge.

Entities are annotated with LOC (location), ORG (organisation), PER (person) and MISC (miscellaneous).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import pandas as pd
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,SimpleRNN,Dropout,LSTM,Bidirectional,Input
from keras.utils.np_utils import to_categorical
from collections import Counter
from keras_crf import CRFModel

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
import matplotlib.pyplot as plt

In [17]:
import importlib
from aula_named_entity_recognition import *
importlib.reload(aula_named_entity_recognition)

<module 'aula_named_entity_recognition' from 'C:\\Users\\guimi\\Documents\\Python_Projects\\DataScience\\NLP\\aula_named_entity_recognition.py'>

In [18]:
ds_train = tfds.load('conll2003', split='train', shuffle_files=True)
ds_valid = tfds.load('conll2003', split='dev', shuffle_files=False)
ds_test = tfds.load('conll2003', split='test', shuffle_files=False)

In [19]:
df_train = preprocessDataFrame(tfds.as_dataframe(ds_train))
df_valid = preprocessDataFrame(tfds.as_dataframe(ds_valid))
df_test = preprocessDataFrame(tfds.as_dataframe(ds_test))

In [20]:
df_train
# type(df_train.iloc[0]['tokens']) = list
# type(df_train.iloc[0]['tokens'][1]) = <class 'str'>

,tokens,ner
0,"["", If, they, 're, saying, at, least, 20, perc...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[Lauck, 's, lawyer, vowed, he, would, appeal, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[Thailand, 's, powerful, military, thinks, the...","[5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, ..."
3,"[A, forensic, scientist, who, examined, the, s...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,"[Werder, Bremen, 3, 0, 1, 2, 4, 6, 1]","[3, 4, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
14037,"["", He, was, not, involved, ...]","[0, 0, 0, 0, 0, 0]"
14038,"["", It, goes, without, saying, that, we, 're, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
14039,"[Bowling, :]","[0, 0]"
14040,"[National, League]","[7, 8]"


In [21]:
int2tag = label2int()

int2tag

{1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC',
 0: 'O'}

In [24]:
tag2int = {}
for key in int2tag:
    value = int2tag[key]
    tag2int[value] = key
print(tag2int)

C:\Program Files\JetBrains\PyCharm 2022.2.3\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_utils.py:606: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in s.iteritems():
C:\Program Files\JetBrains\PyCharm 2022.2.3\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_utils.py:606: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in s.iteritems():
C:\Program Files\JetBrains\PyCharm 2022.2.3\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_utils.py:606: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in s.iteritems():
C:\Program Files\JetBrains\PyCharm 2022.2.3\plugins\python\helpers\pydev\_pydevd_bundle\pydevd_utils.py:606: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for item in s.iteritems():
C:\Program Files\JetBrains\PyCharm 2022.2.3\plug